# Rayleigh-Benard convection

In this tutorial we show how to set up a Rayleigh-Benard convection (RBC) simulation from scratch. The examples folder contains this set up and we hope that this tutorial can deepen the understanding of the workflow.

In [1]:
import json
import subprocess
NEKO_PATH="/home/adperezm/software/neko"

## Create a mesh

For this example we will create a box with origin in 0 and length 1 in all directions. This procedure can be easily done with the genmeshbox script compiled with neko.

The script takes the start and end points of the cube in each of the directions, the number of elements and information on wether the boundaries of the box will be periodic. In this tutorial we use the python subprocess module to execute shell. You can observe how it is done and adapt it to your needs.

In [2]:
lx = 1
ly = 1
lz = 1
nelv_x = 8
nelv_y = 8
nelv_z = 8
periodic_x = '.false.'
periodic_y = '.false.'
periodic_z = '.false.'
command = f"{NEKO_PATH}/contrib/genmeshbox/genmeshbox {0} {lx} {0} {ly} {0} {lz} {nelv_x} {nelv_y} {nelv_z} {periodic_x} {periodic_y} {periodic_z}"

In [3]:
process = subprocess.run(command, capture_output=True, shell=True)
print(process.stdout.decode())

 
    _  __  ____  __ __  ____ 
   / |/ / / __/ / //_/ / __ \
  /    / / _/  / ,<   / /_/ /
 /_/|_/ /___/ /_/|_|  \____/ 
 
 (version: 0.8.99)
 (build: 2024-07-29 on x86_64-pc-linux-gnu using gnu)
 
 Writing data as a binary Neko file box.nmsh
 Done



Great! now you have created a box mesh that can be used for simulations of RBC or any other case.

## Create a case file

The case file contains all the information needed to execute a Neko simulation. It is a json file which of course means that it can be created from a simple directory. We will do so now, going through the important parts in the creation. Detailed information on what are the options can be found [here](https://neko.cfd/docs/develop/dd/d33/case-file.html).

We start by creating an empty directory with the casefile version.

In [4]:
case_file = {'version' : 1.0}
case_file['case'] = {}

Now to the important parts!

First we can add general information about the case we want to run. We start by adding the mesh that we just created. The name of the file can be seen in the output above! Then we add some parameters that will affect every subcomponent of the simulation, as the time to run and the timestep.

In [5]:
case_file['case']['mesh_file'] = 'box.nmsh'
case_file['case']['end_time'] = 50
case_file['case']['timestep'] = 5e-3

### Numerics

On each Neko simulation, we must specify a numerics keyword that references the order in space and time that used in the discretization of the equations to integrate. This information is relevant casewise, as all components of the simulations are affected.

Here we will simply create a subdirectory that we will add to the case file. We are interested in space order 7 and time order 3. We therefore perform dealiasing in the non linear term.

In [6]:
numerics = {'time_order': 3, 'polynomial_order' : 7, 'dealias' : 'true'}
case_file['numerics'] = numerics

### Fluid

The fluid sub-dictionary contains all the information related to the fluid solver, that is, the section of the code that time integrates the momentum equation of Navier-Stokes. This is very likely where most of the information regarding your run will go, so please remember to check all the options  [here](https://neko.cfd/docs/develop/dd/d33/case-file.html).

We start by creating a subdictionary that will contain all the relevant information and set the fluid scheme, which currently in Neko is only the PnPn formulation of navier stokes.

Note that the momentum equation has the following form with the chosen non dimensionalization:

$$ \frac{\partial{\tilde{u}_i}}{\partial{\tilde{t}}} +\tilde{u}_j \frac{\partial{\tilde{u}_i}}{\partial{\tilde{x}_j}} = - \frac{\partial{\tilde{p}}}{\partial{\tilde{x}_i}} + \sqrt{\frac{Pr}{Ra}} \frac{\partial^2{\tilde{u}_i}}{\partial{\tilde{x}_j^2}} + \tilde{T}\delta_{i3}$$

In [7]:
case_file['fluid'] = {}
case_file['fluid']['scheme'] = 'pnpn'

#### Fluid properties

The properties of the fluid are necesary in the NS equations. In typical cases, it is enough to provide the density and the dynamic viscocity. For this particular case, we will use as input parameters the Rayleigh and Prandlt numbers and calculate the viscocity from there. 

In [8]:
case_file['fluid']['Ra'] = 1e6
case_file['fluid']['Pr'] = 1.0

Note that neko allows us to not set the typical parameters in the case file, but by doing this, we are required to provide a user routine that calculates the density and the dynamic viscocity. We will show that in the subsequent sections.

#### Boundary and initial conditions

For the time integration of a PDE we of course need proper boundary and initial conditions, therefore we start by adding those.

In the creation of the mesh, one typically has boundary labels. Here in the casefile, we assign values to said labels. Our box has 6 boundary faces. Face 1-2 are normal to x, 3-4 are normal to y and 5-6 are normal to z. For the velocity in RBC, we will set those to be walls, replicating an experimental convection cell. We can do this easily by setting the following:

In [9]:
boundary_types = ['w','w','w','w','w','w']
case_file['fluid']['boundary_types'] = boundary_types

Note that we set the 6 boundaries to be walls by assigning the proper string to the list index corresponing to each boundary.

Now we set the initial condition. We want the velocity to be zero everywhere, as RBC is a bouyancy driven case. To set this, we can make use of the predefined uniform condition already programmed in Neko. 

We want the uniform initial condition, and we set the 3 velocity components to zero, as shown now:

In [10]:
initial_condition = {'type' : 'uniform', 'value' : [0, 0, 0] }
case_file['fluid']['initial_condition'] = initial_condition

#### Source terms

For this RBC simulation we use the boussinesq approximation for the bouyancy terms. Here we sent that most fluid properties remain constant while the density changes linearly with the temperature. We then introduce the buoyancy effects into the momentum equation through a forcing term and the temperature becomes an active scalar.

In Neko, such term is already programmed as the boussinesq source term. Which takes the temperature (what we call scalar in the case file) calculated in the simulation and applies the following operations:  
$$ \rho \beta (T - T_{ref} )\cdot g $$

Note that in the non normalization of choice, the temperature is added as a forcing term directly, therefore the extra parameters that we choose for the boussinesq term are as follow:

In [11]:
boussinesq_term = {}
boussinesq_term['type'] = 'boussinesq'
boussinesq_term['g'] = [0, 0, 1]
boussinesq_term['reference_value'] = 0
boussinesq_term['beta'] = 1

case_file['fluid']['source_terms'] = [boussinesq_term]

#### Solvers

In the time integration of navier stokes with the pnpn formulation, we must use iterative solvers in two instances: for the velocity and for the pressure.

The type of solver and tolerances are set separately for these. We show here how to set it:

In [12]:
velocity_solver = {'type' : 'cg', 'preconditioner' : 'jacobi', 'projection_space_size' : 5, 'absolute_tolerance' : 1e-6, 'max_iterations' : 800}
pressure_solver = {'type' : 'gmres', 'preconditioner' : 'hsmg', 'projection_space_size' : 20, 'absolute_tolerance' : 1e-4, 'max_iterations' : 800}

case_file['fluid']['velocity_solver'] = velocity_solver
case_file['fluid']['pressure_solver'] = pressure_solver

#### Sampling

The last relevant information to provide is how much we should write fluid fields to disk. We do this by spevifying the following parameters:

In [13]:
case_file['fluid']['output_control'] = 'simulationtime'
case_file['fluid']['output_value'] = 1

### Scalar

For RBC, we need to also solve a transport equation for the Temperature, which becomes coupled with the momentum equation by the afore mentioned forcing term.

The non dimensionalization that we will use is the following:

$$ \frac{\partial{\tilde{T}}}{\partial{\tilde{t}}} +\tilde{u}_j \frac{\partial{\tilde{T}}}{\partial{\tilde{x}_j}} = \frac{1}{\sqrt{RaPr}} \frac{\partial^2{\tilde{T}}}{\partial{\tilde{x}_j^2}}$$

Once again we start by creating an empty dictionary. Here we do not need to set a scheme, no coupled equation with pressure is to be solved.

In [14]:
case_file['scalar'] = {}

#### Fluid properties

In this case we observe that for the non dimensionalization we also need Ra and Pr, which have been previously given in the fluid section. Therefore we do not need to add more information in the scalar, as we will calculate the relevant parameters in the user module. Please read the documentation on the casefile to see typically needed information.

#### Boundary and initial conditions

For this case, we are integrating a PDE so we still need boundary and initial coniditons. In RBC, the top and bottom walls are cooled and heated, respectively, while the sidewalls are adiabatic.

In practice, we set the top and bottom walls with Dirichlet conditions for the temperature, i.e., keeping the temperature constant. While for the side we need Neumman conditions setting the normal derivative of the temperature to zero. In Spectral element methods, the homogeneous Neumman condition is implied in the formulation. Because of this, and following the same approach as before, we set:

In [15]:
boundary_types = ['','','','','d=1','d=0']
case_file['scalar']['boundary_types'] = boundary_types

Here, by doing nothing, we set homogenous Newmann conditions on the sides and we set the temperature at the lower wall to 1 and upper wall to 0.

For the initial condition we will include some perturbation in the temperature to trigger convection. We will define this in the user module, therefore we can use the following input: 

In [16]:
initial_condition = {'type' : 'user'}
case_file['scalar']['initial_condition'] = initial_condition

#### Solvers

For the scalar equations, we use the same solver configuration that is set for the velocity. Therefore no input is needed here.

### Closing remarks!

You are set! Now simply write the case file to disk!

In [17]:
with open('rayleigh.case', 'w') as file:
    file.write(json.dumps(case_file, indent=4))

## User module

In the user module you will be able to set any function that in the case file you set to be user given. In our case there are two instances in which this is true: 

- The material properties.
- The scalar initial condition.

Unfortunately, since this is fortran code, we can not interface nicely in this tutorial. However please refer to the provided rayleigh.f90 file.


### Setting the material properties

In the file, we use one subroutine to set the material properties (Since as mentioned before, we did not provide Re or mu directly)

Note that first we read the provided fluid properties (Ra, Pr). These are not used directly by the code, we need to operate with them to find the relevant mu and lamba that correspond to the non dimensionalization that we have cited here before. 

```fortran
 subroutine set_material_properties(t, tstep, rho, mu, cp, lambda, params)
    real(kind=rp), intent(in) :: t
    integer, intent(in) :: tstep
    real(kind=rp), intent(inout) :: rho, mu, cp, lambda
    type(json_file), intent(inout) :: params
    real(kind=rp) :: Re

    call json_get(params, "case.fluid.Ra", Ra)
    call json_get(params, "case.fluid.Pr", Pr)
    
    Re = sqrt(Ra / Pr)
    mu = 1.0_rp / Re
    lambda = mu / Pr
    rho = 1.0_rp
    cp = 1.0_rp
    
  end subroutine set_material_properties
```

### Setting the scalar initial coniditons

We need one subroutine to set the initial conditions, since this is what we indicated in the case file.

Notice that there are 3 distinct moments in the routine:

- We initially set the temperature to vary linearly from 1 to 0 over the domain.
- We introduce some perturbations to the temperature. In this case we superimpose sine waves at different frequencies.
- We copy the information to the relevant device if there is any.

```fortran
  subroutine set_ic(s, params)
    type(field_t), intent(inout) :: s
    type(json_file), intent(inout) :: params
    integer :: i, e, k, j
    real(kind=rp) :: rand, x, y, z, pert, p1, p2, p3, p4

    do i = 1, s%dof%size()
       s%x(i,1,1,1) = 1-s%dof%z(i,1,1,1)
    end do

    do e = 1, s%msh%nelv
       do k = 2,s%Xh%lx-1
          do j = 2,s%Xh%lx-1
             do i = 2,s%Xh%lx-1

                rand = 1_rp
                x = s%dof%x(i,j,k,e)
                y = s%dof%y(i,j,k,e)
                z = s%dof%z(i,j,k,e)
                p1 = 1_rp*sin(3_rp*pi*x)
                p2 = 0.6_rp*sin(7_rp*pi*x)
                p3 = 0.3_rp*sin(24_rp*pi*x)
                p4 = 0.1_rp*sin(5_rp*pi*x)
                pert = (p1 + p2 + p3 + p4)
                s%x(i,j,k,e) = 1_rp-z + 0.01_rp*pert*rand

            end do
          end do
       end do
    end do

    if ((NEKO_BCKND_DEVICE .eq. 1) .or. (NEKO_BCKND_HIP .eq. 1) &
       .or. (NEKO_BCKND_OPENCL .eq. 1)) then
       call device_memcpy(s%x, s%x_d, s%dof%size(), &
                          HOST_TO_DEVICE, sync=.false.)
    end if


  end subroutine set_ic


```

### Compiling

When we use functions in the user module, it is not possible to use directly the neko executable obtained when it was compiled. We now must compile the user module and use neko with it. This is done with the makeneko script, and we show now how that can be done in practice.

In [ ]:
command = f"{NEKO_PATH}/bin/makeneko rayleigh.f90"

In [ ]:
process = subprocess.run(command, capture_output=True, shell=True)
print(process.stdout.decode())